In [1]:
import pandas as pd
import plotly.express as px

## Download data 
This code download the last two years of data from QU39 and generate and pandas dataframe

In [9]:
# Let's download the last two montsh of data from QU39 in a csv format
url = 'https://catalogue.hakai.org/erddap/tabledap/HakaiWaterPropertiesInstrumentProfileProvisional.csv?&station="QU39"&time>now-2years';

# Load data and ignore the second row which is the units associated with each columns in ERDDAP csv format
df = pd.read_csv(url, skiprows=[1])
df.head()

<ipython-input-9-814feb74c14c>:5: DtypeWarning:

Columns (8,26,29,32,35,38,41,44,47,50,53,56) have mixed types. Specify dtype option on import or set low_memory=False.



,work_area,cruise,hakai_id,vessel,operators,comments,device_model,device_sn,device_firmware,cast_processing_stage,...,c_star_at_flag_description,ph,ph_UQL,cph_flag_description,backscatter_beta,backscatter_beta_UQL,backscatter_beta_flag_description,cdom_ppb,cdom_ppb_UQL,cdom_flag_description
0,QUADRA,QOMA5,18032_2021-04-19T17:42:05.834Z,HAKAI SCOUT,"eva.jordison,meagan.abele",NaN,XRX-620,18032,6.81,10_qc_pi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QUADRA,QOMA5,18032_2021-04-19T17:42:05.834Z,HAKAI SCOUT,"eva.jordison,meagan.abele",NaN,XRX-620,18032,6.81,10_qc_pi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,QUADRA,QOMA5,18032_2021-04-19T17:42:05.834Z,HAKAI SCOUT,"eva.jordison,meagan.abele",NaN,XRX-620,18032,6.81,10_qc_pi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,QUADRA,QOMA5,18032_2021-04-19T17:42:05.834Z,HAKAI SCOUT,"eva.jordison,meagan.abele",NaN,XRX-620,18032,6.81,10_qc_pi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,QUADRA,QOMA5,18032_2021-04-19T17:42:05.834Z,HAKAI SCOUT,"eva.jordison,meagan.abele",NaN,XRX-620,18032,6.81,10_qc_pi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Generate Figures

### Plotly express scatter timeseries

In [12]:
fig = px.scatter(df,x='time',y='depth',color='temperature',range_color=[6,12])
fig.update_yaxes(autorange='reversed')


### Show on a map

In [49]:
import folium
from folium.plugins import FastMarkerCluster

m = folium.Map(
    location= df[['latitude','longitude']].mean(), 
    zoom_start=9, 
    tiles="Stamen Terrain"
)
for name_position,group in df.groupby(['station','latitude','longitude']):
    folium.Marker(
        location=list(name_position[1:]), 
        popup=name_position[0]
    ).add_to(m)

    station_group = group.groupby('hakai_id').first().dropna(axis=0, subset=['precise_latitude','precise_longitude'])
    FastMarkerCluster(
        station_group[['precise_latitude','precise_longitude']].values,
    ).add_to(m)

m